In [2]:
import os
import numpy as np
import torch
import torchani
import ase 
from ase import Atoms
from ase import Atom


In [3]:
device = torch.device('cpu')
path = os.getcwd()
const_file = path + './train/eval/rHCNO-5.2R_16-3.5A_a4-8.params'
sae_file = path + './train/eval/sae_linfit_CAS.dat'
consts = torchani.neurochem.Constants(const_file)
aev_computer = torchani.AEVComputer(**consts)
energy_shifter = torchani.neurochem.load_sae(sae_file)

home = os.getcwd()

In [4]:
mol = []
def printenergy(a=mol):
    epot = a.get_potential_energy()
    ekin = a.get_kinetic_energy()
    return epot + ekin

In [5]:
try:
    os.mkdir('./output/')
except OSError:
    pass

network = './best.pt'

def atomic(linear=[(384,128), (128,128), (128,64), (64,1)]):
    model = torch.nn.Sequential(
        torch.nn.Linear(*linear[0]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[1]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[2]),
        torch.nn.CELU(0.1),
        torch.nn.Linear(*linear[3])
    )
    return model
nn = torchani.ANIModel([atomic(v) for v in (
[(384,160), (160,128), (128,96), (96,1)],
[(384,144), (144,112), (112,96), (96,1)],
[(384,128), (128,112), (112,96), (96,1)],
[(384,128), (128,112), (112,96), (96,1)]
)])

In [6]:
mol = [ Atom('C', (  -0.000000,    0.663585,   -0.000000)),
        Atom('C', (  -0.000000,   -0.663585,    0.000000)),
        Atom('H', (  -0.000000,    1.235740,    0.924684)),
        Atom('H', (  -0.000000,    1.235740,   -0.924684)),
        Atom('H', (  -0.000000,   -1.235740,    0.924684)),
        Atom('H', (  -0.000000,   -1.235740,   -0.924684)) ]

mol = Atoms(mol)

In [7]:
personal_directory = os.getcwd()
calculator = torchani.ase.Calculator(personal_directory, consts.species, aev_computer, network, nn, energy_shifter, device)

mol.set_calculator(calculator)
energy = float(str(printenergy(a=mol))) * (1/units.Hartree) * 627.509474062112 # Hartree to kcal/mol conversion

C:\Users\AMJ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchani\ase.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  cell = torch.tensor(self.atoms.get_cell(complete=True),


PermissionError: [Errno 13] Permission denied: '.'

In [ ]:
print(energy)

In [ ]:
def printxyz(a=mol):
    """Function to print out xyz coordinates"""
    check_out=os.getcwd() + '/output'
    os.chdir(check_out)
    i=0
    file_name = str(i) + '_checkpoint.pdb'
    file_name = str(file_name)
    while (os.path.isfile(file_name)):
        i=i+1
        file_name = str(i)+'_checkpoint.pdb'
        file_name = str(file_name)
    checkpoint_out = str(i) +  '_checkpoint.pdb'
    write(str(checkpoint_out), mol)
    os.chdir('..')